In [1]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_ablation_baselinegp as file

In [2]:
# Args setup
ntrial = 50
n = 100
ntest = 100
d = 5
noise = 1.0
error_samples = 100
gp_samples = 1000

In [3]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="3:0:0",
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

/nfs/ghome/live/danceh/.local/mambaforge/envs/causalgp/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 11111 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35143 instead
  warnings.warn(
Task exception was never retrieved
future: <Task finished name='Task-68' coro=<Client._gather.<locals>.wait() done, defined at /nfs/ghome/live/danceh/.local/mambaforge/envs/causalgp/lib/python3.12/site-packages/distributed/client.py:2385> exception=AllExit()>
Traceback (most recent call last):
  File "/nfs/ghome/live/danceh/.local/mambaforge/envs/causalgp/lib/python3.12/site-packages/distributed/client.py", line 2394, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-36' coro=<Client._gather.<locals>.wait() done, defined at /nfs/ghome/live/danceh/.local/mambaforge/envs/causalgp/lib/python3.12/site-packages/distributed/client.py:2385> exception=AllExit()>
Trac

In [4]:
# Submitting jobs
futures = []
for seed in range(ntrial):
            f = client.submit(file.main,seed,n,ntest,d,noise,
                             error_samples = error_samples, gp_samples = gp_samples)
            futures += [f]

In [5]:
# Check on futures
futures

[<Future: pending, key: main-9a94cd3cef9d5ad3b6b7ca77bbaf0b67>,
 <Future: pending, key: main-ff2c7b769c0ac0e935dff10f097964a8>,
 <Future: pending, key: main-4c530726fe5d10e8bd75e91e08fb4fdd>,
 <Future: pending, key: main-f661aa14e3b997dd57f760fdb79f4f60>,
 <Future: pending, key: main-cb2c3fbd6222f7d9a1c2f38f879af473>,
 <Future: pending, key: main-de0b55702fe4a5df8a38b5af5a0debdc>,
 <Future: pending, key: main-cabb86c13c03e873be1cf848d5a99af4>,
 <Future: pending, key: main-d6e599330de9561077f33ba9ce063ea3>,
 <Future: pending, key: main-cb94354034d428400ace9f23a306a194>,
 <Future: pending, key: main-78c69ab3455b75c03539972e3270ab31>,
 <Future: pending, key: main-3f87aaffae017a602944ff49b825029e>,
 <Future: pending, key: main-55ade0d5be0fa3fc70bc363b537e9e8d>,
 <Future: pending, key: main-d32ba0f8efd5d1f97c072e2d86c88f9b>,
 <Future: pending, key: main-e2a16acb36ff5389970f5f8e971e89ae>,
 <Future: pending, key: main-a56bcc9225a504a625ecae7084f0c637>,
 <Future: pending, key: main-3f5e8bc1be5

In [6]:
# Getting results
results = client.gather(futures)

KeyboardInterrupt: 

In [7]:
# Closing client
client.close()
cluster.close()

In [ ]:
# Saving results
torch.save(obj = results,
           f = "ablation_baselinegp_ntrial={0}_n={1}_d={2}_noise={3}.pt".format(ntrial,n,d,noise))